Team Translation:
Omkar Kadam
Rahul Dikshit
Sheethal Raghavendra
Madhura Aravendekar
Nitnada Manoonphol

In [1]:
# Step 1: Set Up the Environment
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from sacrebleu.metrics import BLEU
from torch.nn import DataParallel
from unidecode import unidecode
from datetime import datetime
from bert_score import score
from tqdm import tqdm
import pandas as pd
import warnings
import logging
import torch
import glob
import os
import os
import glob
import traceback
from datetime import datetime
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn import DataParallel
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# Step 2: Load and Preprocess the Dataset
# Numbers of Rows before droping the null columns - 1920210
# Numbers of Rows after droping the null - 1908920 (11,290)
import pandas as pd
import json

# Function to read and preprocess JSONL file
def read_jsonl(file_path):
  data = []
  with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
      try:
        data.append(json.loads(line))
      except json.JSONDecodeError as e:
        print(f"Skipping line due to JSONDecodeError: {e}")
    return pd.DataFrame(data)

# Load and preprocess the dataset
jsonl_file_path = 'train_new_output.jsonl'  # Replace with the path to your JSONL file

df = read_jsonl(jsonl_file_path)

# Preprocess the DataFrame
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df.dropna().reset_index(drop=True)
df
class TranslationDataset(Dataset):
    def __init__(self, source_sentences, target_sentences, tokenizer, max_length):
        self.source_sentences = source_sentences
        self.target_sentences = target_sentences
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.source_sentences)

    def __getitem__(self, idx):
        source_sentence = self.source_sentences[idx]
        target_sentence = self.target_sentences[idx]

        source_sentence = source_sentence[:self.max_length]
        target_sentence = target_sentence[:self.max_length]

        input_text = f"translate English to German: {source_sentence}"
        target_text = target_sentence

        try:
            input_ids = self.tokenizer.encode(input_text, add_special_tokens=True)
            target_ids = self.tokenizer.encode(target_text, add_special_tokens=True)
        except KeyError:
            print(f"Skipping example with index {idx} due to tokenization error.")
            return None

        return {
            'input_ids': torch.tensor(input_ids),
            'target_ids': torch.tensor(target_ids),
        }

# Prepare the training and validation datasets
train_size = int(0.8 * len(df))
train_df = df[:train_size]
val_df = df[train_size:]

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

# Create the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')  # Use a smaller model

max_length = 256  # Reduce max length if appropriate

# Create the translation datasets
train_dataset = TranslationDataset(train_df['English'], train_df['German'], tokenizer, max_length)
val_dataset = TranslationDataset(val_df['English'], val_df['German'], tokenizer, max_length)

# Create data loaders
batch_size = 16  # Increase batch size if GPU memory allows

def collate_fn(batch):
    batch = [item for item in batch if item is not None]
    input_ids = pad_sequence([item['input_ids'] for item in batch], batch_first=True)
    target_ids = pad_sequence([item['target_ids'] for item in batch], batch_first=True)
    return {
        'input_ids': input_ids,
        'target_ids': target_ids
    }

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=4)

# Fine-tuning configuration
epochs = 3
learning_rate = 2e-5

# Ensure a GPU is available
if not torch.cuda.is_available():
    raise EnvironmentError("CUDA is not available. This script requires a GPU to run.")

# Create and initialize the model
model = T5ForConditionalGeneration.from_pretrained('t5-small')  # Use a smaller model
model.config.gradient_checkpointing = True
device = torch.device("cuda")

# Wrap the model with DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    model = DataParallel(model)

model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# Set the directory to save the model checkpoints
checkpoint_dir = 'Trained_38'
os.makedirs(checkpoint_dir, exist_ok=True)
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Assuming other parts of your code remain the same and only showing relevant parts for saving the model

def save_model(model, tokenizer, directory):
    """
    Saves the model and tokenizer to the specified directory.
    """
    # Create the directory if it does not exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Save the model
    model_path = os.path.join(directory, 'model')
    model.save_pretrained(model_path)
    
    # Save the tokenizer
    tokenizer_path = os.path.join(directory, 'tokenizer')
    tokenizer.save_pretrained(tokenizer_path)

# Example usage after training and evaluation
model_directory = 'Trained_T5_38'
save_model(model, tokenizer, model_directory)


# TensorBoard writer
writer = SummaryWriter(log_dir='38_tensorboard_logs')

# Fine-tuning loop
accumulation_steps = 2  # Adjust if necessary

# Initialize start_epoch in case no checkpoint is loaded
start_epoch = 0

# Load the last saved checkpoint if available
best_val_loss = float('inf')
existing_checkpoints = sorted(glob.glob(os.path.join(checkpoint_dir, 'epoch_*.pt')))
if existing_checkpoints:
    latest_checkpoint = existing_checkpoints[-1]
    checkpoint = torch.load(latest_checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    best_val_loss = checkpoint['val_loss']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    print(f"Loaded checkpoint: {latest_checkpoint} (Epoch {start_epoch}, Best Val Loss: {best_val_loss:.4f})")

# Adjust epochs if resuming training
epochs = start_epoch + epochs

scaler = torch.cuda.amp.GradScaler()  # For mixed precision

for epoch in range(start_epoch, epochs):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")

    model.train()
    total_loss = 0
    accumulation_counter = 0

    train_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}, Current epoch start: {current_time}", leave=False)

    for batch in train_progress:
        input_ids = batch['input_ids'].to(device)
        target_ids = batch['target_ids'].to(device)

        with torch.cuda.amp.autocast():  # For mixed precision
            outputs = model(input_ids=input_ids, labels=target_ids)
            loss = outputs.loss

        loss = loss / accumulation_steps

        if torch.cuda.device_count() > 1:
            loss = loss.mean()

        scaler.scale(loss).backward()

        accumulation_counter += 1
        if accumulation_counter % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps

        train_progress.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / (len(train_loader) * accumulation_steps)
    writer.add_scalar('Loss/train', avg_loss, epoch)

    # Validation
    model.eval()
    val_loss = 0

    val_progress = tqdm(val_loader, desc=f"Validation {epoch+1}/{epochs}, Current epoch start: {current_time}", leave=False)

    with torch.no_grad():
        for batch_idx, batch in enumerate(val_progress):
            try:
                input_ids = batch['input_ids'].to(device)
                target_ids = batch['target_ids'].to(device)

                with torch.cuda.amp.autocast():  # For mixed precision
                    outputs = model(input_ids=input_ids, labels=target_ids)
                    loss = outputs.loss

                val_loss += loss.item()

                val_progress.set_postfix({'Loss': loss.item()})
            except Exception as e:
                print(f"Error in batch {batch_idx} of the validation loop:")
                print(traceback.format_exc())

    val_loss /= len(val_loader)
    writer.add_scalar('Loss/val', val_loss, epoch)

    checkpoint = {
        'epoch': epoch,
        'val_loss': val_loss,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
    }

    checkpoint_filename = f'epoch_{epoch}.pt'
    checkpoint_path = os.path.join(checkpoint_dir, checkpoint_filename)
    torch.save(checkpoint, checkpoint_path)
    print(f"Saved checkpoint: {checkpoint_path}")

writer.close()
def translate_sentence_greedy(sentence):
    input_text = f"translate English to German: {sentence}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    input_ids = input_ids.to(device)

    output = model.generate(input_ids=input_ids, num_beams=1, do_sample=False, max_length=50)
    translated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
    return translated_sentence

def translate_sentence_multinomial(sentence):
    input_text = f"translate English to German: {sentence}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    input_ids = input_ids.to(device)

    output = model.generate(input_ids=input_ids, num_beams=1, do_sample=True, max_length=50)
    translated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
    return translated_sentence

def translate_sentence_beam(sentence, n):
    input_text = f"translate English to German: {sentence}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    input_ids = input_ids.to(device)

    output = model.generate(input_ids=input_ids, num_beams=n, do_sample=False, max_length=50)
    translated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
    return translated_sentence

def translate_sentence_beam_multinomial(sentence, n):
    input_text = f"translate English to German: {sentence}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    input_ids = input_ids.to(device)

    output = model.generate(input_ids=input_ids, num_beams=n, do_sample=True, max_length=50)
    translated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
    return translated_sentence
source_sentence = "Hello, I am Omkar and this is a test."

translated_sentence_greedy = translate_sentence_greedy(source_sentence)
translated_sentence_multinomial = translate_sentence_multinomial(source_sentence)
translated_sentence_beam = translate_sentence_beam(source_sentence, 3)
translated_sentence_beam_multinomial = translate_sentence_beam_multinomial(source_sentence, 3)

print(f"Translated Sentence Greedy: {translated_sentence_greedy}")
print(f"Translated Sentence Multinomial: {translated_sentence_multinomial}")
print(f"Translated Sentence Beam: {translated_sentence_beam}")
print(f"Translated Sentence Beam Multinomial: {translated_sentence_beam_multinomial}")
reference_corpus = val_df['German'].tolist()
candidate_corpus = []

model.eval()

if len(candidate_corpus) <= 0:
    with torch.no_grad():
        for sentence in tqdm(reference_corpus[:10], desc="Generating translations"):
            translated_sentence = translate_sentence_multinomial(sentence)
            candidate_corpus.append(translated_sentence)

    file = open('candidate_corpus.txt', 'w', encoding='utf-8')
    for line in candidate_corpus:
        file.write(line + "\n\n")
    file.close()
candidate_corpus[0], reference_corpus[:len(candidate_corpus)][0]

bleu = BLEU()

bleu_score = bleu.corpus_score(candidate_corpus, [reference_corpus[:len(candidate_corpus)]])
print(f"BLEU score: {bleu_score.score}")

def calculate_bert_score(candidate_corpus, reference_corpus, batch_size=32):
    lowest = min(len(candidate_corpus), len(reference_corpus))
    candidate_corpus = candidate_corpus[:lowest]
    reference_corpus = reference_corpus[:lowest]

    bert_scores = []
    num_batches = len(candidate_corpus) // batch_size
    for i in tqdm(range(num_batches + 1), desc="Calculating BERTScore"):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(candidate_corpus))
        candidates_batch = candidate_corpus[start_idx:end_idx]
        references_batch = reference_corpus[start_idx:end_idx]

        _, _, batch_scores = score(candidates_batch, references_batch, lang='en', verbose=False)
        bert_scores.extend(batch_scores.tolist())

    return bert_scores

# Example usage
warnings.filterwarnings("ignore")
logging.getLogger("transformers").setLevel(logging.ERROR)
bert_scores = calculate_bert_score(candidate_corpus, reference_corpus)
final_bert = sum(bert_scores) / len(bert_scores)
print(f"BERTScore: {final_bert}")

print("Scores:")
print(f"BLEU: {bleu_score.score}")
print(f"BERT: {final_bert}")

/home/jovyan/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Saved checkpoint: Trained_38/epoch_0.pt


Saved checkpoint: Trained_38/epoch_1.pt


Saved checkpoint: Trained_38/epoch_2.pt
Translated Sentence Greedy: Hallo, ich bin Omkar und das ist ein test.
Translated Sentence Multinomial: Hallo, ich bin Omkar und das ist eine prüfung.
Translated Sentence Beam: Hallo, ich bin Omkar und das ist ein test.
Translated Sentence Beam Multinomial: Hallo, ich bin Omkar und das ist ein test.


Generating translations: 100%|██████████| 10/10 [00:02<00:00,  4.83it/s]


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Calculating BERTScore: 100%|██████████| 1/1 [00:19<00:00, 19.74s/it]

BERTScore: 0.9860177397727966
Scores:
BLEU: 82.55759480385746
BERT: 0.9860177397727966
